In [2]:
from scipy import sparse
import csv
import sys
import os
import pandas
import numpy as np
import multiprocessing
import random
from sklearn.externals import joblib
import sklearn.ensemble  

In [23]:
X = []
Y = []
f1=open(unicode('./datasetPersonTab'))
f2=open(unicode('./datasetNNPTab'))
for i, line in enumerate(f1):
    if (i == 0):
        continue
    X.append(line.strip().split('\t')[1])
    Y.append('Person')
size = len(X)
for i, line in enumerate(f2):
    if (i == 0):
        continue
    X.append(line.strip().split('\t')[1])
    Y.append('No')

In [5]:
filename = './DiplomaDataPerson_X_int.joblib.pkl'
X_int = joblib.load(filename)
filename = './DiplomaDataPerson_X_int_test.joblib.pkl'
X_int_test = joblib.load(filename)
filename = './DiplomaDataPerson_RF50_200000.joblib.pkl'
RF = joblib.load(filename)
filename = './DiplomaDataPerson_enc.joblib.pkl'
enc = joblib.load(filename)
#96945

In [6]:
Y_test_result =RF.predict_proba(enc.transform(X_int))

In [24]:
f_n = 0
f_p = 0
t_n=0
t_p=0
f = open('f_p_X_train', 'w')
for i, y in enumerate(Y_test_result):
    y_str = 'No'
    if y[1]>0.9:
        y_str = 'Person'
        Y[i]='Person'
        f.write(str(i))
        f.write(',')
        f.write(X[i])
        f.write(',')
        f.write(str(y[1]))
        f.write('\n')
    if y_str != Y[i]:
        if y_str=='No':
            f_n+=1
            #print X[10000+i]
        else:
            f_p+=1
            
    elif y_str =='No':
        t_n+=1
    else:
        t_p+=1
recall = t_p*1./(t_p+f_n+1)
presision = t_p*1./(t_p+f_p+1)
print recall
print presision
print f_p, f_n, t_p, t_n
F_measure = 2*presision*recall/(presision+recall)
print F_measure   
#recall_set[len(Y_test_result)] = recall
#presision_set[len(Y_test_result)] = presision
#F_measure_set[len(Y_test_result)] = F_measure
f.close()

0.69711810667
0.999986463804
0 32096 73875 502309
0.821527067301


In [27]:
import random
X_train = X_int[:size]
Y_train = Y[:size]
for i in range(size, len(X_int)):
    y = random.uniform(0, 1)
    if Y[i] == 'Person':
        X_train.append(X_int[i])
        Y_train.append('Person') 
    elif y >0.5:
        X_train.append(X_int[i])
        Y_train.append('No') 

In [28]:
size = 96945
random.shuffle(X_int[size:])
X_sparse_matrix = enc.transform(X_int[:200000])

In [ ]:
import sklearn.ensemble  
RF = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=3)
RF.fit(X_sparse_matrix, Y_train[:200000])